In [222]:
import pandas as pd
import numpy as np
import seaborn as sns



In [223]:
db = pd.read_csv("nba_shotchartdetail_2018-19.csv")
db.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,21800001,7,203496,Robert Covington,1610612755,Philadelphia 76ers,1,11,40,...,Center(C),24+ ft.,26,-53,264,1,0,20181016,BOS,PHI
1,Shot Chart Detail,21800001,10,1628369,Jayson Tatum,1610612738,Boston Celtics,1,11,15,...,Left Side Center(LC),24+ ft.,25,-148,207,1,0,20181016,BOS,PHI
2,Shot Chart Detail,21800001,14,1627759,Jaylen Brown,1610612738,Boston Celtics,1,11,3,...,Center(C),Less Than 8 ft.,1,4,18,1,0,20181016,BOS,PHI
3,Shot Chart Detail,21800001,17,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,10,55,...,Center(C),Less Than 8 ft.,0,-8,3,1,1,20181016,BOS,PHI
4,Shot Chart Detail,21800001,19,1628369,Jayson Tatum,1610612738,Boston Celtics,1,10,36,...,Center(C),Less Than 8 ft.,4,-46,1,1,0,20181016,BOS,PHI


In [224]:
db = db.drop(["GRID_TYPE", "GAME_EVENT_ID", "TEAM_ID", "SHOT_ATTEMPTED_FLAG"],1)
db.head()

/tmp/ipykernel_11067/1091249216.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  db = db.drop(["GRID_TYPE", "GAME_EVENT_ID", "TEAM_ID", "SHOT_ATTEMPTED_FLAG"],1)


,GAME_ID,PLAYER_ID,PLAYER_NAME,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,21800001,203496,Robert Covington,Philadelphia 76ers,1,11,40,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,26,-53,264,0,20181016,BOS,PHI
1,21800001,1628369,Jayson Tatum,Boston Celtics,1,11,15,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,25,-148,207,0,20181016,BOS,PHI
2,21800001,1627759,Jaylen Brown,Boston Celtics,1,11,3,Missed Shot,Running Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,4,18,0,20181016,BOS,PHI
3,21800001,203954,Joel Embiid,Philadelphia 76ers,1,10,55,Made Shot,Running Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-8,3,1,20181016,BOS,PHI
4,21800001,1628369,Jayson Tatum,Boston Celtics,1,10,36,Missed Shot,Driving Layup Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,4,-46,1,0,20181016,BOS,PHI


In [225]:
def hit():   
    return db[db["SHOT_MADE_FLAG"] == "1"]

def actiontype(actiontype):
    return db[db["ACTION_TYPE"] == actiontype]

In [227]:
def hitrate(playername):
    a = db[db['PLAYER_NAME'] == playername]
    return a.groupby('PLAYER_NAME').SHOT_MADE_FLAG.mean()

def hitrateShottype(playername, shottype):
    a = db[db['PLAYER_NAME'] == playername]
    b = a[db['SHOT_TYPE'] == shottype]
    return b.groupby('PLAYER_NAME').SHOT_MADE_FLAG.mean()

def hitrateShottype_Action(playername, actiontype, shottype):
    a = db[db['PLAYER_NAME'] == playername]
    b = a[db['ACTION_TYPE'] == actiontype]
    c = b[db['SHOT_TYPE'] == shottype]
    return c.groupby('PLAYER_NAME', 'ACTION_TYPE', 'SHOT_TYPE').SHOT_MADE_FLAG.mean()

hitrateShottype_Action ("Jayson Tatum", "Jump Shot", "Jump Shot")



PLAYER_NAME
Jayson Tatum    0.449807
Name: SHOT_MADE_FLAG, dtype: float64

In [ ]:
def hitrateTeam(teamname):
    a = db[db['TEAM_NAME'] == teamname]
    return a.groupby('TEAM_NAME').SHOT_MADE_FLAG.mean()

def hitrateTeamShottype(teamname, shottype):
    a = db[db['TEAM_NAME'] == teamname]
    b = a[db['SHOT_TYPE'] == shottype]
    return b.groupby('TEAM_NAME').SHOT_MADE_FLAG.mean()

hitrateTeamShottype("Boston Celtics", "3PT Field Goal")

/tmp/ipykernel_11067/4120920332.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  b = a[db['SHOT_TYPE'] == shottype]


TEAM_NAME
Boston Celtics    0.364793
Name: SHOT_MADE_FLAG, dtype: float64

In [ ]:

def hitratebytime():
    a=(db["PERIOD"]*12)-db["MINUTES_REMAINING"]
    db.insert(7,"GAME_MINUTES_PLAYED", a)
    db1 = db[db['PERIOD'] <= 4]
    db2 = db[db['PERIOD'] > 4]
    b = (db2["PERIOD"]-4) * 5 - db2['MINUTES_REMAINING'] + 48
    db2 = db2.drop(['GAME_MINUTES_PLAYED'], 1)
    db2.insert(7,"GAME_MINUTES_PLAYED", b)
    dball = db1.append(db2)
    return dball.groupby("GAME_MINUTES_PLAYED").SHOT_MADE_FLAG.mean()

def hitratebytime_player(playername, time = 100):
    a=(db["PERIOD"]*12)-db["MINUTES_REMAINING"]
    db.insert(7,"GAME_MINUTES_PLAYED", a)
    db1 = db[db['PERIOD'] <= 4]
    db2 = db[db['PERIOD'] > 4]
    b = (db2["PERIOD"]-4) * 5 - db2['MINUTES_REMAINING'] + 48
    db2 = db2.drop(['GAME_MINUTES_PLAYED'], 1)
    db2.insert(7,"GAME_MINUTES_PLAYED", b)
    dball = db1.append(db2)
    dball = dball[dball['PLAYER_NAME'] == playername]
    if time != 100:
        dball = dball[dball['GAME_MINUTES_PLAYED'] == time]
    return dball.groupby("GAME_MINUTES_PLAYED").SHOT_MADE_FLAG.mean()

def hitratebytime_team(teamname, time = 100):
    a=(db["PERIOD"]*12)-db["MINUTES_REMAINING"]
    db.insert(7,"GAME_MINUTES_PLAYED", a)
    db1 = db[db['PERIOD'] <= 4]
    db2 = db[db['PERIOD'] > 4]
    b = (db2["PERIOD"]-4) * 5 - db2['MINUTES_REMAINING'] + 48
    db2 = db2.drop(['GAME_MINUTES_PLAYED'], 1)
    db2.insert(7,"GAME_MINUTES_PLAYED", b)
    dball = db1.append(db2)
    dball = dball[dball['TEAM_NAME'] == teamname]
    if time != 100:
        dball = dball[dball['GAME_MINUTES_PLAYED'] == time]
    return dball.groupby("GAME_MINUTES_PLAYED").SHOT_MADE_FLAG.mean()
